In [20]:
# import project

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama


In [21]:
MODEL = "llama3.2"

# class for the website
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        

In [22]:
system_prompt = "You are an assistant that analyzes the offers of two websites \
and provides a short summary for both sites, ignoring text that might be navigation related. \
Respond with the offers from both sites."

In [23]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website1, website2):
    user_prompt = f"You are looking at two websites titled {website1.title}, {website2.title}"

    # 1st Website 
    user_prompt += f"### website one: {website1.title}\n"
    user_prompt += "The contents of this website are as follows;"
    user_prompt += "please provide a short summary of this website in markdown."
    user_prompt += "If it includes news or annoucements, then summarize these too.\n\n"
    user_prompt += website1.text + "\n\n"

    # 2nd website
    user_prompt += f"### Website two: {website2.title}\n"
    user_prompt += "The contents of this website are as follow;"
    user_prompt += "please provide a short summary of this website in markdown."
    user_prompt += "If it includes news or annoucements, then summarize these too.\n\n"
    user_prompt += website2.text + "\n\n"

    
    # Compare the websites
    user_prompt += "Finally, compare the offers, announcements, or any relevant information "
    user_prompt += "From both websites and highlight key differences and provide which option is better from the two carriers.\n"

    return user_prompt
    


In [24]:
# See how this function creates exactly the format above

def messages_for(website1, website2):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website1, website2)}
    ]

In [25]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url1, url2):
    website1 = Website(url1)
    website2 = Website(url2)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website1, website2)
    )
    return response['message']['content']

In [26]:
# A function to display this nicely in the Jupyter output, using markdown

def displaySummary(url1, url2):
    # Generate the combined summary for both websites
    combined_summary = summarize(url1, url2)
    
    # Display the combined summary in Markdown format
    display(Markdown(combined_summary))

In [28]:
displaySummary("https://www.att.com/buy/phones/apple-iphone-16-pro.html",
    "https://www.t-mobile.com/cell-phone/apple-iphone-16-pro"
)

After reviewing the content from both AT&T and T-Mobile websites, I have identified some key similarities and differences in their upgrade and trade-in policies for customers:

**Similarities:**

* Both carriers offer a Yearly Upgrade benefit that allows customers to trade-in their device and apply the credit towards a new device.
* Both carriers pay off the remaining balance of the old device (50% of the original purchase price) when a customer upgrades or trades-in their device.

**Differences:**

1. **Eligibility:** T-Mobile's Yearly Upgrade benefit requires devices to be in good condition, while AT&T does not have such a requirement.
2. **Trade-in Value:** AT&T offers more generous trade-in values for certain devices, while T-Mobile's trade-in value is lower for some models.
3. **Promotions:** T-Mobile occasionally runs promotions that allow customers to pay off their EIP balance (Equipment Installment Plan) and receive additional credit towards a new device. These promotions are not available on AT&T's website.
4. **Device Eligibility:** AT&T allows devices from all carriers to be upgraded, while T-Mobile only supports devices from AT&T.

**Comparison of Offers:**

* T-Mobile:
	+ Pay off 50% of the remaining EIP balance when trading-in a device
	+ No requirements for good condition or specific trade-in values
	+ Occasional promotions that allow customers to pay off their EIP balance and receive additional credit
* AT&T:
	+ Pay off 50% of the remaining EIP balance when trading-in a device (no trade-in value requirement)
	+ Higher trade-in values for certain devices
	+ No promotions available on their website

**Which Option is Better?**

T-Mobile's Yearly Upgrade benefit is more flexible and offers more options for customers who want to upgrade or trade-in their device. However, the trade-in value may be lower compared to AT&T's offerings. If you're a T-Mobile customer and want to take advantage of this program, it's worth exploring their promotions and eligibility requirements.

On the other hand, if you're an AT&T customer and are looking for more generous trade-in values or have devices from all carriers, AT&T may be a better option.

**Recommendation:**

Consider your specific needs and device choices when deciding between T-Mobile and AT&T's upgrade and trade-in policies. If you value flexibility and occasional promotions, T-Mobile might be the better choice. However, if you're looking for more generous trade-in values or want to take advantage of a wider range of devices, AT&T may be a better fit.

Ultimately, it's essential to review both carriers' offers and consider your individual circumstances before making a decision.